### __1. Import Library__

In [69]:
from keras.datasets import mnist

In [70]:
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Reshape, Flatten, LeakyReLU
from tensorflow.keras.optimizers import Adam


### __2. Mendefinisikan Variabel untuk Neural Network dan Data__

In [71]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0001)

### __3. Membentuk Generator__

In [72]:
def build_generator():
    
    model = Sequential()
    
    model.add(Dense(265, input_dim = latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    return model

In [73]:
generator = build_generator()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 265)               26765     
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 265)               0         
                                                                 
 batch_normalization_15 (Bat  (None, 265)              1060      
 chNormalization)                                                
                                                                 
 dense_36 (Dense)            (None, 256)               68096     
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_16 (Bat  (None, 256)              1024      
 chNormalization)                                    

### __4. Mendefinisikan Discriminator__

In [74]:
def build_discriminator():
    
    model = Sequential()
    
    model.add(Flatten(input_shape = img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(256))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    return model


In [75]:
discriminator = build_discriminator()
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_39 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_40 (Dense)            (None, 256)               131328    
                                                                 
 dense_41 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


### __5. Menghubungkan Discriminator dengan Generaotr untuk membentuk GAN__

In [76]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss = 'binary_crossentropy', optimizer =adam)

GAN.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_15 (Sequential)  (None, 28, 28, 1)         365249    
                                                                 
 sequential_16 (Sequential)  (None, 1)                 533505    
                                                                 
Total params: 898,754
Trainable params: 363,695
Non-trainable params: 535,059
_________________________________________________________________


### __6. Outputting Images__

In [77]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

In [78]:
save_name = 0.00000000

In [79]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

### __7. Training GAN__

In [80]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  # X_train = np.expand_dims(X_train, axis=3)
  # print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    #Get Random Batch
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss[0] if isinstance(g_loss, list) else g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(15000, batch_size=64, save_interval=200)

2/2 [==============================] - 0s 9ms/step
******* 0 [D loss: 0.979796, acc: 39.84%] [G loss: 0.702291]
1/1 [==============================] - 0s 236ms/step
0.00000001
saved
2/2 [==============================] - 0s 5ms/step
******* 1 [D loss: 0.260356, acc: 93.75%] [G loss: 0.756389]
2/2 [==============================] - 0s 9ms/step
******* 2 [D loss: 0.227159, acc: 92.19%] [G loss: 0.988993]
2/2 [==============================] - 0s 6ms/step
******* 3 [D loss: 0.207160, acc: 91.41%] [G loss: 1.293038]
2/2 [==============================] - 0s 5ms/step
******* 4 [D loss: 0.247890, acc: 86.72%] [G loss: 1.928892]
2/2 [==============================] - 0s 7ms/step
******* 5 [D loss: 0.229174, acc: 86.72%] [G loss: 2.604542]
2/2 [==============================] - 0s 8ms/step
******* 6 [D loss: 0.194665, acc: 90.62%] [G loss: 3.271582]
2/2 [==============================] - 0s 7ms/step
******* 7 [D loss: 0.035214, acc: 100.00%] [G loss: 4.071151]
2/2 [============================

### __8. Making GIF__

In [81]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

C:\Users\Ega\AppData\Local\Temp\ipykernel_6380\1295497862.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
C:\Users\Ega\AppData\Local\Temp\ipykernel_6380\1295497862.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
